# AutoEncoder
 



## 0.환경준비

### 0.1 라이브러리 로딩

In [ ]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.backend import clear_session
from tensorflow.keras.utils import plot_model

### 0.2 필요 함수들 생성

#### ① reconstruction error plot

> * input과 output의 차이(mse)를 계산하고
* 시각화 합니다.



In [ ]:
def recon_err_plot(x, x_pred, y, threshold=0):
    # treshold : 우리가 지정해줘야 함.(어떻게?)
    
    mse = np.mean(np.power(x - x_pred, 2), axis=1)
    error_df = pd.DataFrame({'Reconstruction_error': mse, 'True_class': y})
    error_df = error_df.reset_index()

    groups = error_df.groupby('True_class')
    fig, ax = plt.subplots()
    for name, group in groups:
        ax.plot(group.index, group.Reconstruction_error, marker='o', ms=3.5, linestyle='',
                label= "Abnormal" if name == 1 else "Normal")
    ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
    ax.legend()
    plt.title("Reconstruction error for different classes")
    plt.ylabel("Reconstruction error")
    plt.xlabel("Data point index")
    plt.show()

    return error_df

#### ② precision, recall, f1 curve

> * sklearn에서는 precision, recall curve만 제공됩니다. 
* 그래서, f1 curve도 추가해서 구하고, plot을 그립니다.



In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

def prec_rec_f1_curve(y, score, pos = 1) :
    precision, recall, thresholds  = precision_recall_curve(y, score, pos_label=1)
    f1 = 2 / (1/precision + 1/recall)

    plt.plot(thresholds, np.delete(precision, 0), label = 'precision')
    plt.plot(thresholds, np.delete(recall, 0), label = 'recall')
    plt.plot(thresholds, np.delete(f1, 0), label = 'f1')
    plt.xlabel('Threshold')
    plt.legend()
    plt.grid()
    plt.show()

    return precision, recall, f1, thresholds

#### ③ threshold로 잘랐을 때, 분류 평가 함수


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def classification_report2(y, pred, thresholds):
    pred_temp = np.where(pred > thresholds , 1, 0)

    cm = confusion_matrix(y, pred_temp)
    print('< confusion matrix >\n')
    print(cm)
    print('\n' + '='*60 + '\n')

    print('< classification_report >\n')
    print(classification_report(y, pred_temp))

    return cm

## 1.AE 연습 : 직원 이직 예측

* 일부 데이터를 뽑아서 AutoEncoder를 만들어 봅시다.

### 1.1 데이터 준비

#### 1) 데이터 로딩

In [ ]:
# data data
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/Attrition2.csv"
data = pd.read_csv(path, usecols = ['Attrition', 'Age', 'DistanceFromHome', 'MaritalStatus', 'MonthlyIncome'])
data['Single'] = np.where(data['MaritalStatus'] == 'Single', 1, 0)
data.drop('MaritalStatus', axis = 1, inplace = True)

data.rename(columns={"DistanceFromHome": "Distance", "MonthlyIncome": "Income"}, inplace = True)
data.head(10)

#### 2) x, y 나누기 

In [ ]:
target = 'Attrition'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

In [ ]:
y.value_counts() / y.shape[0]

#### 3) 데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# train_val에서 train : val = 8 : 2
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state = 2022)

In [ ]:
print(x_train.shape, x_val.shape)

#### 4) 스케일링

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

#### 5) 학습용 데이터 준비
* 학습에는 x_train만 사용됩니다.(비지도)
* 또한, x_train 중에서도 Normal 데이터만 이용합니다.

In [ ]:
# 학습을 위해서는 Normal 데이터만 이용한다.
x_train0 = x_train_s[y_train == 0]

In [ ]:
x_train0.shape

### 1.2 모델링 : AE

#### 1) hyper parameter 설정

In [ ]:
epochs = 50
input_dim = x_train0.shape[1] #num of columns
input_dim

tf.random.set_seed(10)

#### 2) 모델 설계

![](https://github.com/DA4BAM/dataset/blob/master/ae_1.png?raw=true)

In [ ]:
clear_session()

input_layer = Input(shape=(input_dim, ))

# Encoder
encoder = Dense(4, activation="relu")(input_layer)
encoder = Dense(2, activation="relu")(encoder)

# Decoder
decoder = Dense(4, activation='relu')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.summary()

#### 3) compile + 학습
* 학습시 x=x_train0, y=x_train0를 입력합니다. (비지도)

In [ ]:
# Configure the learning process, by compiling the model
autoencoder.compile(optimizer='adam',loss='mse', metrics=['accuracy'])

history = autoencoder.fit(x=x_train0, y=x_train0,
                          epochs=epochs,
                          validation_data=(x_val_s, x_val_s)).history

In [ ]:
plt.plot(history['loss'], label='Train')
plt.plot(history['val_loss'], label='Validation')

plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid()
plt.show()

### 1.3 예측 및 Reconstruction Error 계산

* 예측

In [ ]:
pred = autoencoder.predict(x_val_s)

* 재구성 에러 계산

    * recon_err_plot(y, y_pred, threshold)
        * threshlod는 임의로 지정하는 값입니다.
    * 그래프에서
        * x 축 : 각 데이터(instance, 행) 인덱스
        * y 축 : 각 데이터(instance, 행) - 예측값과의 재구성 에러
        * 파란색점과 주황색점은 실제값을 나타냅니다.
        * 빨간선 : 임의로 지정한 threshold(cutoff)

In [ ]:
result = recon_err_plot(x_val_s, pred, y_val, .03)

In [ ]:
result.head()

* threshold(cutoff) 값을 조정하면서 precision, recall, f1 을 구해봅시다.
    * threshold(cutoff) 보다 크면, **Abnormal**
    * prec_rec_f1_curve(y, recon_error)
        * x축 : 재구성에러(Anomaly Score라고 부르기도 함)
        * y축 : threshold값을 0부터 조금씩 조정하며 증가시켰을 때의 f1, recall, precision 값(0~1)

In [ ]:
precision, recall, f1, thresholds = prec_rec_f1_curve(result['True_class'], result['Reconstruction_error'])

* f1 값을 최대화 시키는 threshold 값

In [ ]:
thres_f1_max = thresholds[np.argmax(f1)]
thres_f1_max

* threshold 값에 따른 classification report
    * classification_report2(y, Reconstruction_error , threshold)

In [ ]:
classification_report2(result['True_class'], result['Reconstruction_error'],thres_f1_max)

In [ ]:
# 수용 할 만한 결과인가요?


### 1.4 [옵션] deep dive into reconstruction error
아래 내용은 시간이 허용되는 만큼 진행하고자 합니다.

* reconstruction error를 직접 계산해보고
* reconstruction error에 영향을 가장 많이주는 변수 분석 


#### 1) 실제값과 예측값 합치기

In [ ]:
# 실제값을 데이터프레임으로 저장
x_val_s = pd.DataFrame(x_val_s, columns = list(x))

# 예측값을 데이터프레임으로 저장(칼럼명에 prefix : 'Pr_')
pred_col = []
for i in list(x):
    pred_col.append('Pr_'+i)
pred_df = pd.DataFrame(pred, columns = pred_col)

# 합치기
reconErr = pd.concat([x_val_s, pred_df], axis = 1)
reconErr.head()

#### 2) reconstruction error 직접 계산

* 각 값끼리의 Squared Error를 구하기.

In [ ]:
for v in list(x) :
    v_se, v_pred = 'SE_'+v, 'Pr_'+v
    reconErr[v_se] = np.power(reconErr[v] - reconErr[v_pred], 2)

reconErr.head()

* 각 행에 대한 mse 계산

In [ ]:
reconErr['Recon_Error'] = (reconErr['SE_Age'] + reconErr['SE_Distance'] + reconErr['SE_Income'] + reconErr['SE_Single'])/4
reconErr.head()

#### 3) reconstruction error에 영향을 많이 주는 변수
* 만약, 모델이 적절(?)하다고 판단될 때,
* 모델에 의해 계산된 재구성 에러가 큰 데이터(instance, 행)는 Abnormal 이라고 볼 수 있다.
* 그렇다면, 재구성 에러 계산시 값이 큰 변수가 이상탐지에 중요한 변수로 볼 수 있다.
* 그래서 변수별 mse를 계산해보자.

In [ ]:
for v in list(x) :
    print(v.ljust(10, ' '), round(np.mean(reconErr['SE_'+v]),4), round(np.std(reconErr['SE_'+v]),4))

## 2.실습 : Semiconductor manufacturing process dataset


![](https://assets.pandaily.com/uploads/2021/10/semiconductor.png)

* 반도체 제조 공정은 시점별로 수많은 센서로부터 정보를 수집하여 공정을 감시합니다. 
* 센서정보와 함께 공정간 발생된 불량품에 대한 정보를 저장하였습니다.
* 불량을 예측해 봅시다.

### 2.1 데이터 준비

* 데이터 로딩

In [ ]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/secom_9.csv"
data = pd.read_csv(path)

data['label'] = 0
data.loc[data['defeat']== 'defeat', 'label']= 1
data.drop(['datetime','defeat'], axis = 1, inplace=True)
data.head()

변수 정보 
* label : 1 - 불량, 0 - 정상
* v### : 센서값들


In [ ]:
target = 'label'

In [ ]:
data[target].value_counts() / data.shape[0]

* x, y로 나누기 

In [ ]:
x = data.drop(target, axis = 1)
y = data.loc[:, target]

* 가변수화 Dummy Variable

* 데이터 분할
    * 이미 test set은 분할되어 있다고 가정합니다.
    * 주어진 데이터를 train set : validation set 으로 분할

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# train_val에서 train : val = 8 : 2
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state = 2022)

In [ ]:
print(x_train.shape, x_val.shape)

* 스케일링(Optional)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

* 학습용 데이터 준비

In [ ]:
# 학습을 위해서는 Normal 데이터만 이용한다.


### 2.2 모델링①

* 간단한 hyper parameter 설정
    *   epochs, batch size 등 설정


* 모델 설계 : 다음 그림과 같이 설계하시오.

![](https://github.com/DA4BAM/dataset/blob/master/ae_2.png?raw=true)

* compile + 학습
    * 학습시 x=x_train0, y=x_train0를 입력합니다. (비지도)

* 예측

* 재구성 에러 계산

* threshold 값을 조정하면서 precision, recall, f1 을 구해봅시다.

* f1 값을 최대화 시키는 threshold 값

* classification_report2 로 평가

----

### 2.3 모델링②

* hyper parameter 설정

* 모델 설계 : 다양한 구조를 설계하시오.

* compile + 학습
    * 학습시 x=x_train0, y=x_train0를 입력합니다. (비지도)

* 예측

* 재구성 에러 계산

* threshold 값을 조정하면서 precision, recall, f1 을 구해봅시다.

* f1 값을 최대화 시키는 threshold 값

* classification_report2 로 평가